## SteadyFlow: Spatially Smooth Optical Flow for Video Stabilization

## Abstract

우리는 SteadyFlow라는 stablization을 위한 새로운 motion representation을 제안합니다. SteadyFlow 는 특정한 optical flow로써 강한 spatial coherence를 강요합니다. 이는 smoothing feature trajectories를 smootihg pixel profile로 대체합니다. smooting pixel profile은 같은 pixel에서 시간동안 모아진 motion vector입니다. 이를 통해 비디오 stabilization 시스템에서 feature tracking을 피할 수 있습니다. 또한 SF는 spatially-variant motion을 다룰 수 있는 일반적인 2d motion 모델입니다. 우리는 SF를 optical flow를 통해 초기화 하고 spatial-temporal 분석을 통해 불연속적인 motion을 제거합니다. 그 후 motion completion을 통해 없는 region을 채우게 됩니다. 실험을 통해 우리의 시스템이 실제 비디오에서 효율적인 것을 입증합니다.

## 1. Introduction

비디오 안정화는 adopted motion model에 매우 의존적입니다. 몇몇 방법들은 연속적인 프레임 사이에 파라미터화된 2d motion model(homography or mixture homography) 를 가정합니다. 이러한 방법들은 로버스트 하지만 spatially variant motion을 다루기엔 제한적입니다. feature trajectories은 보다 복잡한 non-parametric 2d motion representation을 제공합니다. 최근의 연구들은 smoothing feature trajectories을 통해 좋은 안정화 성능을 달성했습니다. 그러나 feature trajectories을 다루는 것은 매우 어렵습니다. feature trajectories은 공간적으로 매우 sparse하며 고르게 분포하지 않습니다. 또한 보다 긴 trajectories를 얻는 것은 consumer(아마추어?) 비디오에서 얻는 것은 빠른 카메라 회전, motion blur 때문에 어렵습니다.

optical flow와 같은 dense 2d motion field는 보다 유연하고 강력한 motion model입니다. optical flow가 공간적으로 smooth 할때 우리는 smoothing feature trajectories를 smoothing pixel profile로 근사할 수 있습니다. pixel profile이란 같은 pixel location에서의 시간에 따른 motion vector입니다. 다른 말로 하면 우리는 smoothing feature trajectories를 smooth pixel profile로 대체할 수 있습니다. 더이상 특징 추적이 필요하지 않습니다. 모든 pixel profile은 첫번째 프레임에서 시작해 마지막 프레임에서 끝나게 됩니다. 이는 consumer video의 안정화에서 바람직한 특징입니다.

일반적인 비디오에서의 optical flow는 보다 불연속적이며 특히 강한 depth edge와 움직이는 객체에서 많이 불연속적입니다. 그러므로 optical flow를 SF로 수정하는것이 필요합니다. SF는 optical flow의 close approximation이며 강한 공간적 smoothness를 부여함으로써 원래의 비디오를 안전화 하기 위해 SF에서 추출된 pixel profile을 간단히 smooth 할 수 있습니다. 우리는 SF를 전통적인 optical flow로 초기화 하고 spatial-temporal 분석을 통해 불연속 적인 motion vector를 찾아냅니다. 이러한 불연속 적인 flow는 삭제되고 없어진 region을 motion completion을 통해 채우게 됩니다. 실험을 통해 우리의 방법의 로버스트함을 보여줍니다.

## 2. RelatedWork

비디오 안정화 기술은 보통 2d, 3d 방법으로 나눠집니다. 2d 안정화 방법은 camera motion을 나타내기 위해 homography나 affine model의 sequence를 사용합니다. 그 후 비디오를 안정화 하기 위해 이러한 sequence들을 smooth 합니다. 초기의 2d 방법들은 적절한 motion smoothing 기술을 설계하느데 초점을 뒀습니다. 몇몇 방법들은 간단한 Gaussian smoothing을 사용합니다. Chen은 camera motion을 표현하기 위해 polynomial curve를 피팅시켰습니다. Gleicher는 개별적인 smoothing을 위해 카메라 궤적을 segment로 나눕니다. Grundmann은 camera path smoothing을 위해 L1-norm 최적화를 적용합니다. 최근에는 LIu가 mixure-of-homography motion model을 추정하기 위한 'as-similar-as-possible' warping을 적용합니다. 2d 방법들은 보통 로버스트하지만 움직이는 객체가 있거나 불연속적인 depth 가 있는 복잡한 장면을 표현하는데 한계점이 있습니다. 우리는 3d 안정화 방법들과 같은 non-linear motion을 다루기 위해 per-pixel flow를 사용하는 더 일반적인 2d motion model을 적용합니다. 

3d 방법들은 안정화를 위해 3d camera motion을 추정합니다. Beuhler는 uncalibrated camera에서 projective 3d reconstruction을 통한 안정화 기술을 설계합니다. Liu는 'content-preseving' warping을 통한 3d 안정화 시스템을 개발합니다. Zhou는 여기에 plane constraint를 추가했습니다. LIu는 로버스트한 안정화를 위해 depth camera를 사용합니다. 일반적으로 이러한 3d 방법들은 좋은 성능을 보이며 특히 non-trivial depth change인 장면에서 성능이 좋습니다. 그러나 3d reconstruction을 위해선 depth sensor와 같은 것들이 필요합니다. 최근의 방법들은 이러한 것들을 long feature tracks에 있는 부분적인 3d 정보들을 활용하여 완화합니다. Liu는 안정화를 위해 부분공간에서의 feature 궤도의 기저들을 smooth 합니다. Wang은 각각 궤도들을 Bezier 곡선으로 표현하고 이를 spatial-temporal 최적화를 통해 smooth 합니다. Glodstein은 long feature track의 변형을 완화하기 위해 epipolar tranfer를 사용합니다. smoothing feature track은 light-field camera를 안정화 하기 위해서도 사용됩니다. 최근에는 Liu 가 입체 비디오를 다루기 위해 subspace 방법을 확장합니다. 대부분의 3d 방법들이 로버스트한 feature tracking을 필요로 합니다. long feature trajectories를 얻는 것은 어렵습니다. 우리는 tracking을 요구하지 않는 smmothing pixel profile을 통해 smoothing feature tracjectories의 효과를 근사합니다. 


## 3. SteadyFlow Model

이 섹션에서는 pixel profile의 개념을 소개합니다. 우리는 SF의 pixel profile을 smoothing하는 것이 어떻게 비디오를 안정화 할 수 있는지를 설명합니다. 그 후 우리는 SF 모델을 설명하고 feature tracjecctories에 비해 어떠한 장점이 있는지를 보여줍니다.

## 3.1. Pixel Profiles vs. Feature Trajectories

pixel profile은 같은 픽셀에서의 motion vector의 집합으로 구성되어져 있습니다. feature tracjectory는 scene point(특징점?을 의미하나?)의 motion을 따라 갑니다. 피규어1 은 t-1 frame 에서의 a pixel 에서 시작하는 pixel profile, feature tracjectory를 보여줍니다. feature trajectory는 t-1 프레임에서 A 픽셀의 움직임을 따라 t 프레임의 B 픽셀로 가고 t+1 프레임에서의 C 픽셀을 따라갑니다. pixel profile은 시간에 따라 A 픽셀의 motion을 모읍니다. 

## 3.2. Stabilization by Smoothing Pixel Profile

간단한 예제에서 시작해 봅시다. 피규어 2는 점진적으로 depth가 변하는 장면에서의 비디오를 보여줍니다. 이 장면에서의 otpical flow는 오른쪽에 보이는 것처럼 공간적으로 부드럽습니다. 우리는 같은 픽셀에서 추출된 pixel profile을 간단하게 부드럽게 합니다. 이 방법을 통해 안정화된 비디오를 얻을 수 있습니다. 이는 smoothing pixel profile 을 통해 비디오를 안정화 할 수 잇다는 것을 의미합니다.

이것에 대한 이해를 위해 공개된 데이터셋의 108개의 비디오에 대해서 실험하였습니다. 이 108개 비디오의 모든 연속적인 프레임에대해서 optical flow를 계산합니다. 또한 좋은 feature trajectories를 얻기 위해 KLT tracker를 실행합니다. 또한 Adobe After Effect CS6 Roto brush가 지원하는 비디오에서의 움직이는 객체들을 수동으로 표시합니다. 이 방법을 통해 60 프레임보다 적은 길이의 배경에 대한 14662 trajectories와 움직이는 객체에 대한 5595 tracjectories를 얻었습니다. 우리는 feature trajectoy와 pixel profile을 통해 얻은 궤도를 비교합니다. 이 차이는 feature trajectories, pixel profile을 통해 얻은 motion vector의 평균을 통해 평가됩니다. 피규어3은 두 방법을 통해 얻어진 궤적의 차이에 대한 히스토그램을 보여줍니다. 피규어3.a 는 정적 배경에서의 feature trajectories의 90% 정도가 pixel profile과 비슷하다는 것을 보여줍니다. 이는 smoothing feature trajectories가 smoothing pixel profile을 통해 근사될 수 있음을 보여줍니다. 피규어 3.b는 움직이는 객체에서의 feature trajectories와 pixel profile의 차이를 보여줍니다. 

## 3.3. SteadyFlow

피규어 3.b에 대한 분석은 pixel profile이 feature trajectories와 많이 다를 수 있음을 보여줍니다. 피규어 4.a 4.c는 이러한 문제를 다루기 위해 복잡한 optical flow를 가지고 있는 두 비디오를 보여줍니다. 피규어에서 볼 수 있듯이 flow vector는 움직이는 사람과 강한 depth edge의 경우 불연속적이 됩니다. raw optical flow에 대한 pixel profile을 부드럽게 한다면 클로즈업된 피규어에서 볼 수 있듯이 이미지의 왜곡이 발생할 것입니다. 이는 불연속적인 flow에서는 smoothing pixel profile이 안좋은 결과를 보인다는 것을 나타냅니다.

우리는 raw optical flow를 수정하여 SF를 얻으려고 합니다. SF는 두가지 특징을 충족해야 합니다. 첫번째로 이는 raw optical flow와 비슷해야 하며 두번째로는 이미지의 왜곡을 피하기 위해 공간적으로 부드러워야 합니다. 이러한 특징이 만족된다면 SF를 통해 얻어진 pixel profile을 부드럽게 함으로써 비디오를 안정화 할 수 있습니다. 피규어 4.b, 4.d를 통해 SF로 얻어진 smoothing pixel profile의 결과를 보여줍니다. 이 결과는 왜곡으로 부터 자유롭습니다.

raw optical flow에 대한 간단한 Gaussian smoothing은 불충분합니다. smoothing은 움직이는 객체의 motion을 background로 전달할 수 있음으로 이는 움직이는 객체 근처에서의 시간적으로 울렁거림을 생성하고 frame registration accuracy를 떨어뜨리게 됩니다. 대신에 우리는 불연속적인 flow vector를 찾아내어 제거한뒤 이를 SF의 두가지 특징을 만족하도록 채워넣습니다. 상세사항은 섹션 4.2, 4.3에서 기술합니다. 

## 3.4. Advantages over Feature Trajectories

비디오 안정화에서 pixel profile이 feature trajectories보다 좋은 몇가지 이유가 있습니다. 첫번째로 pixel profile은 공간적,시간적으로 dense합니다. 반면 feature trajectories는 sparse하며 균등하게 분포하지 않으며 비디오 프레임 밖으로 벗어날 수 있습니다. 이 떄문에 feature tracjectoreis를 부드럽게 하기 위한 필터를 설계하기가 어렵게 됩니다. 두번째로는 긴 feture track을 정확하게 얻는 것이 매우 어렵습니다. frame by frame tracing optical flow를 통해 dense feature track을 얻을 수 있지만 이러한 feature track은 많은 drifting error를 가지게됩니다. 세번쨰로 smootthing feature track은 몇가지 왜곡을 초래할 수 있습니다. 추가적인 조건 (subspace projection) 등이 더 나은 smoothing을 위해 필요로 합니다. 비교를 위해 flow filed가 공간적으로 부드럽더라도 pixel profile이 개별적으로 smooth 할 수 잇다는 것을 보입니다.

pixel profile은 optical flow의 품질에 의존적입니다. optical flow estimation은 종종 객체의 경계나 텍스쳐가 없는 region에서 부정확합니다. 대부분의 경우 텍스쳐가 없는 region은 적은 구조를 가지고 있기 때문에 적은 왜곡을 초래하게 됩니다. 객체의 경계에서의 부정확한 flow는 불연속 flow 제거와 motion completion을 통해 완화됩니다.


## 4. SteadyFlow Estimation

우리의 비디오 안정화 시스템의 파이프라인은 피규어5에 나와있습니다. 먼저 SF를 로버스트한 optical flow를 통해 초기화 합니다. 공간적 smooth를 강제하기 위해 불연속적인 motion vector를 찾아내고 이를 인접한 pixel의 motion vector의 interpolation을 통해 채워넣습니다. 그 후 pixel profile 을 기반으로한 안정화가 적용됩니다.  우리는 iterative 한 방법을 통해 SF esmtiamtion의 성능을 증가시킵니다. 최종 결과는 안정화된 SF에 따라 랜더링 되게 됩니다.

## 4.1. Initialization

우리는 SF를 로버스트한 optical flow 추정을 통해 초기화 합니다. 먼저 두 프레임 사이의 KLT feature matching을 통해 global homography transformation을 추정합니다. 우리는 두 프레임을 transfromation에 따라 정렬하고 liu가 제안한 optical flow를 적용합니다. SF는 residual optical flow의 합과 global homography에 의한 motion displacement에 의해 초기화 됩니다. 

## 4.2. Discontinuity Identification

다른 motion 을 탐지하기 위해 가능한 솔루션중 하나는 motion segmentation 기술을 사용하는 것입니다. 그러나 motion segmentation은 그 자체로 매우 어려운 문제이며 많은 방법들이 long feature track을 필요로 합니다. 두 프레임에서 작동하는 몇몇 방법드리 있지만 전형적으로 이들은 큰 fg object 가 있는 경우 인접한 프레임 사이의 motion contrast가 크지 않아 어렵습니다. 

우리는 불연속적인 flow vector를 찾기 위한 새로운 spatial-temporal analysis를 도입합니다. 이를 통해 탐지된 pixel들은 일종의 outlier pixel로 여겨지게 됩니다. 우리는 outlier mask Mt(p) 를 통해 이를 나타내며 t 프레임의 p pixel이 아웃라이어면 0 아니면 1을 사용합니다. 공간적 도메인에서 불연속적인 region을 찾기 위해 raw optical flow의 gradient magnitude를 사용합니다. gradient magnitude가 역치보다(<b>실험에서는 0.1</b>) 를 넘게 되면 그 픽셀이 outlier로 고려합니다. 공간적인 분석은 움직이는 객체의 경계 pixel만을 찾아낼 수 있습니다. 이는 움직이는 객체안의 motion vector는 비슷한 경우가 많고 background 와의 motion만 다른 경우가 많기 떄문입니다. 그러므로 객체 안의 것들도 잘 찾기 위해 temporal 분석이 필요합니다.

temporal analysis는 누적된 motion vector를 $c_t(p) = \sum_t{u_t(p)}$ 살펴보고 outlier를 판단하게 됩니다. 이는 안정적인 비디오에서는 누적 motion vector가 강한 edge나 움직이는 객체를 제외하고는 시간적으로 부드럽다는 관찰에 기반을 합니다. 피규어6은 안정화된 비디오와 두 픽셀 에서의 누적 motion vector를 보여줍니다. 하얀색 별로 마킹된 픽셀은 배경에 대한 픽셀입니다. 이 픽셀에서 누적된 motion vetor는 피규어 6.b에서 볼 수 있듯이 시간적으로 부드러운 track을 생성하게 됩니다. 반면에 피규어 6.a에서 볼 수 잇듯이 하얀색 점으로 마킹된 픽셀에서의 누적된 motion vector는 초기에 높은 주파수를 보입니다. 처음 몇 프레임에서 하얀색 점에 해당하는 곳에 사람이 움직였기 떄문입니다. 사람이 멀리 갔을때 이 픽셀의 track이 smooth하게 되게 됩니다. 우리는 outlier mask를 다음과 같이 계산합니다. 여기서 G는 가우시안 필터이며 (<b>std가 3을 사용</b> epsilon은 적응적인 threshold를 사용합니다. 

## $ M_t(p) = \left\{\begin{matrix}
0, \,\, ||c_t(p) - G \bigotimes c_t(p)|| > \epsilon
\\ 
1,\,\, otherwise
\end{matrix}\right.$

원래 신호 - low-passed signal = high-passed signal?

어떻게 구현? 그냥.. 누적이미지를 다 구해서.. 여기에 필터링 한다음 역치 해주면 될 것 같다. 메모리 효율적으로 짜고 싶다면.. 이미지 하나만큼의 메모리만 가지고 한칸씩 움직이면서 하면 될 듯?

## 4.3. Motion Completion

outerlier mask로 부터 모든 불연속적인 motion을 모읍니다. mask에 속하게 되는 motion vector들은 제거됩니다. 그 후 [17] 과 비슷한 방법을 통해 채워넣게 되는데 이는 'as-similar-as-possible' warping을 통해 수행됩니다. 근본적으로 mask boundary를 control point(?) 로 사용하고 40x40 mesh grid를 warping하여 motion을 채우게 됩니다. 수학적으로는 이는 $E(V) = E_d(V) + E_s(V) $ 를 최소화 하는 것과 같습니다. [14]에서 기술것과 같은 smoothness term E_s를 사용했으며 이는 grid의 rigidity를 유지합니다. 데이어 텀은 다음과 같이 정의됩니다.

## $ E_d(V) = \sum_p{M(p) \cdot ||V \pi_p - (p + u_p)||}$

여기서 grid vertice는 V로 표시됩니다. vector u_p 는 pixel p 에서의 초기 optical flow로써 control point로 부터 (p, p+u_p)가 됩니다..(???) 파라미터 pi_p는 blinear coordinate이며 

피규어 7은 추정된 SF를 보여줍니다. 피규어 7.c에서 하얀색 region으로 보이는 부분은 움직이는 객체, depth edge에 해당하는 부분과 부정확한 raw opticalf low를 나타냅니다. 사라진 region의 motion vector는 근처의 픽셀로 부터 interpolation이 됩니다. 이 방법을 통해 우리는 피규어 7.d와 같은 SF를 생성할 수 있습니다. 

raw optical flow는 Gaussian filter를 통해서도 부드러워질 수 있습니다. 그러나 Gaussian smooth는 전경의 motion을 배경으로 전파합니다. 이는 배경의 frame registration(프레임을 정확히 위치시키는것?) 을 실패하게 하며 안정화된 비디오에서의 시간적으로 울렁거림을 유발합니다.

## 4.4. Iterative Refinement

outlier mask를 추정하기 위한 temporal analysis는 안정적인 비디오를 필요로 합니다. 피규어 6.c, 6.d에서 볼 수 있듯이 원래의 흔들린 비디오에서 생성된 tracjectories는 불연속 적입니다. 실무적으로 우리는 초기의 outlier mask는 spatial analysis만을 사용해 추정합니다. 그후 iterative한 방법을 통해 outlier mask를 정교화 합니다. 각각의 반복마다 첫번째 단계는 outlier를 제거하고 missing region을 mask 에 따라 채우게 됩니다. motion completion은 섹션 4.3에서 기술된 대로 진행합니다. 두번째 단계는 SF를 안정화 하는 것입니다. 이는 섹션 5에서 기술됩니다. 세번째 단계는 안정화된 SF는 섹션 4.2에서 기술된 temporal analysis를 사용하여 outlier mask를 더 정교하게 합니다. temporal analysis는 안정화된 비디오에서 사용하는 것이 더 적절함으로 우리는 epsilon을 adaptive 하게 적용합니다. 이 세가지 단계를 반복적으로 수행하고 안정화된 비디오를 생성하게 됩니다. 우리는 경험적으로 총 5회의 iteration을 수행합니다. 

## 5. Pixel Profiles based Stabilization

이 섹션에서는 SF를 통해 추출된 pixel profile을 smooth하는 비디오 안정화 알고리즘을 도출합니다. U_t를 프레임 t-1, t 를 가지고 추정한 원래 비디오에서의 SF라고 하고 S_t를 안정화된 비디오에서의 SF라고 합시다. pixel smoothing은 다음과 같은 objective function을 최소화 함으로써 달성되게 됩니다. 

## $O(\{P_t\}) = \sum_t\begin{pmatrix}
||P_t - C_t||^2 + \lambda \sum_{r \in \Omega_t} w_{t,r} ||P_t - P_r ||^2
\end{pmatrix}$

$C_t = \sum_t U_t$ 는 Input video의 누적된 motion vector이며 $P_t = \sum_t S_t$ 입니다. 첫번째 텀은 안정화된 비디오와 가깝도록 하며 이는 과도한 cropping을 피하도록 합니다. 두번쨰 텀은 temporal smoothness를 강요하게 됩니다. 

첫번째 텀은 원래의 누적 motion vector와 차이가 별로 안나게 하며 두번째 텀은 temporal window 만큼의 gaussian weight,, 즉 시간적으로 인접한 누적 motion vector끼리 별로 차이가 안나게 하고 싶다!! 

[18]에서 언급된 path optimization 방법과의 3가지 차이점이 있습니다. 첫번째로 SF 자체가 강한 공간적 smoothness를 가지므로 수식 3에 추가적인 공간적 smoothness 제약이 필요 없습니다. 두번째로 w_t,r 웨이트는 bilateral weight가 아닌 오직 spatial Gaussian function만을 포함합니다. 다른 motion 강도를 적응적으로 다루기 위해 우리는 smoothing과정에서 adaptive toemporal window를 사용합니다. 세번째로 P,C 는 homography와 같은 파라미터화 된 모델이 아닌 누적 motion vector를 사용하는 non-parametric 모델이 됩니다.

우리는 다음과 같은 iterative solution을 얻을 수 있습니다. 

## $ P_t^{\xi + 1} = \frac{1}{\gamma} (C_t + \lambda \sum_{r \in \Omega_t, t\neq r} w_{t,r}P_r^{(\xi )})$

여기서 scalar $\gamma = 1+ \lambda \sum_r w_{t,r}$ 이며 xi는 iteration index이며 우리는 10을 사용합니다. 최적화 후에 원래의 비디오를 dense flow field $B_t = P_t - C_t$(안정화된 누적 motion vector - 원래 비디오에서의 누적 motion vector = 원래 비디오에서 안정화된 비디오로의 motion vector?) 를 사용하여 안정화된 프레임으로 warping합니다. 다음과 같은 관례를 도출 할 수 있습니다.

## $U_t + B_{t-1} = B_t + S_t => S_t = U_t + B_{t-1} - B_t$

밑의 식으로 이해하면 위에가 더 잘 이해될듯
## $B_t - B_{t-1} = (P_t - P_{t-1}) - (C_t - C_{t-1}) = S_t - U_t $

원래 비디오의 누적 motion vector + 이전 프레임에서의 안정화 비디오 - 원래 비디오의 motion vector = 안정화된 비디오에서의 누적 motion vector + 현재 프레임에서의 안정화 비디오 - 원래 비디오의 motion vector 

결국.. t 프레임의 안정화된 프레임을 얻기 위해서 이전 프레임을 S_t가지고 warping 시키면.. 안정화된 프레임을 얻을 수 있을 것 같다. 

# 5.1. AdaptiveWindow Selection

우리의 smoothing 기술은 temple window 안에서 feature trajectories와 pixel profile이 비슷하다는 것을 필요로 합니다. 비디오에서의 motion velocity의 변화를 다루기 위해서 temple window를 적응적으로 조절하는 것이 필요합니다. 피규어 8에서 볼 수 있듯이 SF는 윈도우 안에서 공간적으로 부드럽다고 가정됩니다. 윈도우는 픽셀 A를 중심으로 (2t+1)x(2t+1) 의 크기를 가집니다. 이 window 안에서 red line으로 표시된 feature trajectories를 smoothing 하는 것은 dash line으로 표시된 pixel profile을 smoothing하는 것으로 근사 할 수 있습니다. trajectory가 window를 벗어나게 된다면 (우리의 실험에서는 tau = 20 을 사용) 근사된 에러는 매우 클 것입니다. 이를 위해 pixel profile의 각 pixel에 대해 각각 window를 추정하며 이를 통해 모든 프레임에서 feature trajectory가 (2t+1)x(2t+1) window에 들어오도록 해줍니다. feature trajectory는 tracing optical flow를 통해 근사될 수 있습니다. 예를 들어 피규어 8에서  A 픽셀에 대한 적절한 window는 Omega_t(A) = [t-1, t+r] 그림에서보면 바로 전 프레임과 +r 프레임까지 trajectory가 window안에 들어옴. 공간적 왜곡을 피하기 위해서 프레임 t에서의 모든 픽셀에 대해한 global smooth window를 선택하는것은 필수적입니다. 우리는 모든 픽셀에서의 window에 대해 교집합을 취했고 이를 가지고 temporal support를 결정했습니다. 이러한 동적인 윈도우를 통해 빠른 카메라 모션에서의 비디오를 다룰 수 있습니다.